<p>It is advisable that you read our introductory documentation webpage before moving on with understading the code. As it would help you understand the problem better.</p>
<p>You can check it out <a href="https://hasocfire.github.io/hasoc/2021/ichcl/index.html">here</a></p>

### Importing Libraries and initializing stopwords and stemmer

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import stemmer as hindi_stemmer

<p>Making a list of english and hindi stopwords. <br>The enlgish stopwords are retrieved from NLTK library as well. <br>And the hindi stopwords are retrieved from a data set on Mendeley Data. To read about how the authors compiled the list, you can check their <a href = "https://arxiv.org/ftp/arxiv/papers/2002/2002.00171.pdf" > publicaion </a> </p>
<p>Initializing an english SnowballStemmer using the NLTK library. <br>And the hindi stemmer used was produced by students of Banasthali University. You can check out their <a href="https://arxiv.org/ftp/arxiv/papers/1305/1305.6211.pdf">publication</a></p>

In [2]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

## Reading Data

<p>Initializing a list of various directories that data is stored in using the glob Library.</p>

In [3]:
train_directories = []
for i in glob("data/train/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [4]:
train_directories

['data/train\\bantwitter\\1397101600460529665\\',
 'data/train\\bantwitter\\1397235232621932545\\',
 'data/train\\bantwitter\\1397245294597783556\\',
 'data/train\\bantwitter\\1397895650218430465\\',
 'data/train\\bantwitter\\1397914911431217152\\',
 'data/train\\bantwitter\\1397923242938015749\\',
 'data/train\\bantwitter\\1397959669696634885\\',
 'data/train\\casteism\\1391715849979924484\\',
 'data/train\\casteism\\1394701718458310661\\',
 'data/train\\casteism\\1394894310781321218\\',
 'data/train\\casteism\\1394920395438886917\\',
 'data/train\\casteism\\1394929122237845509\\',
 'data/train\\casteism\\1395690830292160520\\',
 'data/train\\casteism\\1395747405786537988\\',
 'data/train\\casteism\\1395768553190604804\\',
 'data/train\\casteism\\1397952483532566536\\',
 'data/train\\casteism\\1398675045266853888\\',
 'data/train\\charlie hebdo\\1392703278022938626\\',
 'data/train\\charlie hebdo\\1392704604387770374\\',
 'data/train\\charlie hebdo\\1392715113853964288\\',
 'data/trai

<p>Reading tree structured data from the directories from the .json files</p>

In [5]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

</p>Defining 2 functions that will turn the data from a tree structure to a flat structure.</p>
<ul>
    <li>tr_flatten: This is to flat the train data. It takes two variables as function parameters. First one is the tweet data and second one is labels. It'll create a list of json structures like following:
        <ul>
            <li> for source tweet: It'll create json with tweet_id, tweet text and label. </li>
            <li> for comment: It'll create json with tweet_id, label and for the text part it'll append the comment after the source tweet. This is a basic technique to provide context of source tweet. </li>
            <li> for reply: It'll create json with tweet_id, label and for the text part it'll append the reply after the comment after the source tweet. So the text here will look like "source tweet-comment-reply"</li>
        </ul>
    </li>
    <li>te_flatten: This is to flat the test data. It works similarly like tr_flatten but without the labels file, as labels won't be available for test set. It'll be used once the test data is available</li>
</ul>

In [6]:
def tr_flatten(d,l):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
        'label':l[d['tweet_id']]
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] +' '+i['tweet'], #flattening comments(appending one after the other)
                'label':l[i['tweet_id']]
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] +' '+ i['tweet'] +' '+ j['tweet'], #flattening replies
                        'label':l[j['tweet_id']]
                    })
    return flat_text

def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + i['tweet'] + j['tweet'],
                    })
    return flat_text

<p>This cell will run both the flatten functions. Again, you can skip the test part if it is not available. The train_len variable will be used later on for splitting the data.</p>

In [7]:
data_label = []
#for train
for i in range(len(labels)):
    for j in tr_flatten(data[i], labels[i]):
        data_label.append(j)
train_len = len(data_label)

In [8]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [9]:
df.head()

,tweet_id,text,label
0,1397101600460529665,Countries which have Banned Twitter\n\n🇨🇳 Chin...,HOF
1,1397101827116703744,Countries which have Banned Twitter\n\n🇨🇳 Chin...,NONE
2,1397101939674869763,Countries which have Banned Twitter\n\n🇨🇳 Chin...,HOF
3,1397102700173488133,Countries which have Banned Twitter\n\n🇨🇳 Chin...,HOF
4,1397102906004754433,Countries which have Banned Twitter\n\n🇨🇳 Chin...,HOF


In [10]:
df['label'].value_counts()

NONE    2899
HOF     2841
Name: label, dtype: int64

In [11]:
tweets = df.text
y = df.label

## Preprocessing and featuring the raw text

<p>This is a preprocessing function and the regex will match with anything that is not English, Hindi and Emoji.</p>
<p>The preprocessing steps are as followed:</p>
<ul>
    <li>Remove Handles</li>
    <li>Remove URLs</li>    
    <li>Remove anything that is not English, Hindi and Emoji</li>    
    <li>Remove RT which appears in retweets</li>    
    <li>Remove Abundant Newlines</li>    
    <li>Remove Abundant whitespaces</li>    
    <li>Remove Stopwords</li>
    <li>Stem English text</li>
    <li>Stem Hindi text</li>
</ul>

In [12]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            token = hindi_stemmer.hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [13]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

<p>Using TF-IDF for featuring the text. The vectorizer will only consider vocab terms that appear in more than 5 documents.</p>
<p>To learn more about TF-IDF you can check <a href = "https://towardsdatascience.com/tf-term-frequency-idf-inverse-document-frequency-from-scratch-in-python-6c2b61b78558">here</a> and <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">here</a>.</p>

In [14]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()

## Training and evaluating model

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

<p>Training the Logistic Regression classifier provided by Scikit-Learn library.</p>
<p>To learn more about Logistic Regression classifier you can check <a href = "https://www.youtube.com/watch?v=yIYKR4sgzI8">here</a> and <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">here</a>.</p>

In [16]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

<p>Predicting and priting classification metrics for validation set.</p>

In [17]:
y_pred = classifier.predict(X_val)

In [18]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.73      0.70      0.72       577
        NONE       0.71      0.74      0.73       571

    accuracy                           0.72      1148
   macro avg       0.72      0.72      0.72      1148
weighted avg       0.72      0.72      0.72      1148



In [19]:
le = LabelEncoder() #label encoding labels for training Dense Neural Network
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

In [20]:
model = Sequential(
    [
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(1, activation="sigmoid"),
    ]
)
model.compile('adam', loss='binary_crossentropy', metrics = ['accuracy']) #compiling a neural network with 3 layers for classification

In [21]:
model.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5
144/144 [==============================] - 0s 2ms/step - loss: 0.6278 - accuracy: 0.6699
Epoch 2/5
144/144 [==============================] - 0s 2ms/step - loss: 0.5244 - accuracy: 0.7485
Epoch 3/5
144/144 [==============================] - 0s 2ms/step - loss: 0.4523 - accuracy: 0.7966
Epoch 4/5
144/144 [==============================] - 0s 2ms/step - loss: 0.3852 - accuracy: 0.8341
Epoch 5/5
144/144 [==============================] - 0s 2ms/step - loss: 0.3311 - accuracy: 0.8643


In [22]:
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype('int64')
y_pred = y_pred.reshape(len(y_pred))    

In [23]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71       577
           1       0.71      0.70      0.70       571

    accuracy                           0.70      1148
   macro avg       0.70      0.70      0.70      1148
weighted avg       0.70      0.70      0.70      1148



## Predicting test data and making a sample submission file

<p>This part will be used to read and make predictions on the test data once the it is made available. When it is available, make a directory in data directory as 'test' and copy the story direcotries into the test directory.</p>

In [ ]:
test_directories = []
for i in glob("data/test/*/"):
    for j in glob(i+'*/'):
        test_directories.append(j)

In [ ]:
test_directories

<p>The test directories do not contain labels.json file so labels list is not initialized for test data.</p>

In [ ]:
test_data = []
for i in test_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))

<p>Flattening the test data.</p>

In [ ]:
test_tweetid_data = []
#for test
for i in range(len(labels), len(data)):
    for j in te_flatten(data[i]):
        test_tweetid_data.append(j)

In [ ]:
test_df = pd.DataFrame(test_tweetid_data, columns = test_tweetid_data[0].keys(), index = None)

In [ ]:
test_df.head()

In [ ]:
test_tweets = test_df.text
tweet_ids = test_df.tweet_id

In [ ]:
cleaned_test = [clean_tweet(tweet) for tweet in test_tweets]

In [ ]:
X_test = vectorizer.transform(cleaned_test)
X_test = X_test.todense()

In [ ]:
submission_prediction = classifier.predict(X_test)
submission = {'tweet_id': tweet_ids, 'label':submission_prediction}
submission = pd.DataFrame(submission)

In [ ]:
submission.to_csv('data/sample_submission.csv', index = False)